In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
datadir = '/content/drive/My Drive/Capstone_traces' 
os.chdir(datadir)

Mounted at /content/drive


In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

NameError: ignored

In [6]:
assert tf.test.is_gpu_available()


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [5]:
assert tf.test.is_built_with_cuda()

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import tensorflow as tf
import time
import datetime


In [ ]:
df = pd.read_csv("trace.log",sep=' ',header=None)
df.columns = ['sender', 'heartbeat_num','send_time','receive_time','hop_left']
df.head()

,sender,heartbeat_num,send_time,receive_time,hop_left
0,7,1,1.096546e+16,2.439203e+15,14.0
1,7,2,1.096546e+16,2.439203e+15,14.0
2,8,0,2.954178e+16,2.439203e+15,16.0
3,7,3,1.096546e+16,2.439203e+15,14.0
4,8,1,2.954178e+16,2.439203e+15,16.0


In [ ]:
df_cut = df[:20000]
df_1 = df_cut[df_cut['sender'] == 1]
df_1.head(),df_1.shape

(     sender  heartbeat_num     send_time  receive_time  hop_left
 809       1              0  4.781611e+16  2.439214e+15      16.0
 818       1              1  4.781611e+16  2.439214e+15      16.0
 827       1              2  4.781611e+16  2.439214e+15      16.0
 836       1              3  4.781611e+16  2.439214e+15      16.0
 845       1              4  4.781611e+16  2.439214e+15      16.0, (2135, 5))

In [ ]:
def scale(dataset):
    scaler = StandardScaler()
    dataset = dataset.reshape(-1,1)
    dataset = scaler.fit_transform(dataset)
    return dataset

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back+1): 
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        if i!=len(dataset)-look_back:
            dataY.append(dataset[i + look_back, 0])

    return np.array(dataX), np.array(dataY)

In [ ]:
import keras.backend as K
def customLoss(true,pred):
    diff = pred - true

    greater = K.greater( K.cast(0, K.floatx()),diff)
    greater = K.cast(greater, K.floatx()) #0 for lower, 1 for greater
    greater = greater * 10000                 #1 for lower, 2 for greater
    greater = greater + 1

    return K.mean(greater*K.square(diff))

In [ ]:
############ THINGS TO CHANGE #############
n = 500 #(100,500,1000)
look_back = 1 #(1,3,10,100)
batch_size = 64 #(32,64)
epochs = 5 #(5,10,...)
############ THINGS TO CHANGE #############

In [ ]:
model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(4, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])
model.compile(loss=customLoss, optimizer='adam')

In [1]:


interval = np.ndarray(0)
interval_scaled = np.ndarray(0)
expected_arrival = [0 for i in range(len(df_1)+1)]
arrival = [0 for i in range(len(df_1))]
diffs = []



sus_sum = 0
k = -1

# delta = 100000000
init_safety_margin = 2000000
freshness_points = [0 for i in range(len(df_1)+1)]


t = time.time()

for i in range(len(df_1)):

    k = i
    rece_time = df_1['receive_time'].values[i]

    arrival[i] = rece_time
    
    

    if k > n+1 and rece_time > freshness_points[i]:
        sus_sum += 1
        diff = rece_time - freshness_points[i]
        diffs.append(diff)
        safety_margin = init_safety_margin + sum(diffs[-10:])/len(diffs[-10:])
        print(k,safety_margin/1000000)
    else:
        safety_margin = init_safety_margin
        


    if k <= n: # for the first n points, just accumulate the results
        expected_arrival[k+1] = rece_time + 100000000 
        if k == 0:
            pass
        else:
            interval = np.append( interval, rece_time - arrival[k-1])
            interval_scaled = scale( interval )



    else: # now, we have three elements in interval array, let's start training

        # calculate the interval between most recent receive and second most
        interval = np.append( interval, rece_time - arrival[k-1])
        interval = interval[-n:] # only keep the n most recent

        scaler = StandardScaler()
        interval_scaled = interval.reshape(-1,1)
        interval_scaled = scaler.fit_transform(interval_scaled)

#         print(interval_scaled,len(interval_scaled))
        # split train's X and Y
        trainX, trainY = create_dataset(interval_scaled, look_back)

        testX = trainX[-1]
        testX = testX.reshape(1,look_back)
        testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

        trainX = trainX[:-1]
        trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))


#         print(trainX,testX)
        # fit lstm, create the model
#         model = 0

        model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=False)


        # use the model to predict
        testY = model.predict(testX)[0]
        testY = scaler.inverse_transform(testY)

#         print(interval[k-1],testY,rece_time)
#         print(testY)
        expected_arrival[k+1] = np.ravel(rece_time)[0] + np.ravel(testY)[0]
        model.reset_states()


        freshness_points[k+1] = expected_arrival[k+1] + safety_margin

        if i>0:
          print(trainX, trainY)
          break        


t2 = time.time()
computation_time_per_iter = ((t2-t)/len(df_1))*1000
pa = 1-(sus_sum)/(len(df_1)-n)
df_1['expected_arrival'] = expected_arrival[:-1]
df_1['freshness_points'] = freshness_points[:-1]
df_1['over_estimation'] = df_1['expected_arrival'] - df_1['receive_time']
df_1['over_estimation_fresh'] = df_1['freshness_points'] - df_1['receive_time']
df_2 = df_1[(df_1['over_estimation'] > 0)== True]
df_2 = df_1[(df_1['over_estimation_fresh'] > 0)== True]
over_estimation_mean = (df_2['over_estimation'].mean())/1000000
over_estimation_fresh_mean = (df_2['over_estimation_fresh'].mean())/1000000

print("The pa is",pa)
print("The over estimation average is", over_estimation_mean,"milliseconds")
print("The over estimation average fresh is",over_estimation_fresh_mean,"milliseconds")
print("The computation time is",computation_time_per_iter,"milliseconds")

                


        
        

NameError: ignored

In [ ]:
###Accumulated Safety Margin on 2000000 points###
# The pa is 0.999445650667529
# The over estimation average is 9.395403540563029 milliseconds
# The over estimation average fresh is 25.56356176942634 milliseconds
# The computation time is 63.36064631930215 milliseconds

In [ ]:
a = [1,2,3]
a[-100:]